### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="l4s-" + fablib.get_bastion_username()

node_conf = [
 {'name': "tx0",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "tx1",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "router", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "delay", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "rx0",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "rx1",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}
]
net_conf = [
 {"name": "net-tx0", "subnet": "10.0.0.0/24", "nodes": [{"name": "tx0",   "addr": "10.0.0.100"}, {"name": "delay", "addr": "10.0.0.101"}]},
 {"name": "net-tx1", "subnet": "10.0.1.0/24", "nodes": [{"name": "tx1", "addr": "10.0.1.100"}, {"name": "delay", "addr": "10.0.1.101"}]},
 {"name": "net-delay-router", "subnet": "10.0.2.0/24", "nodes": [{"name": "delay",   "addr": "10.0.2.1"}, {"name": "router", "addr": "10.0.2.2"}]},
 {"name": "net-rx0", "subnet": "10.0.3.0/24", "nodes": [{"name": "rx0",   "addr": "10.0.3.100"}, {"name": "router", "addr": "10.0.3.101"}]},
 {"name": "net-rx1", "subnet": "10.0.4.0/24", "nodes": [{"name": "rx1", "addr": "10.0.4.100"}, {"name": "router", "addr": "10.0.4.101"}]}

]
route_conf = [
 {"addr": "10.0.3.0/24", "gw": "10.0.0.101", "nodes": ["tx0"]}, 
 {"addr": "10.0.4.0/24", "gw": "10.0.1.101", "nodes": ["tx1"]}, 
 {"addr": "10.0.3.0/24", "gw": "10.0.2.2", "nodes": ["delay"]},
 {"addr": "10.0.4.0/24", "gw": "10.0.2.2", "nodes": ["delay"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.3.101", "nodes": ["rx0"]},
 {"addr": "10.0.1.0/24", "gw": "10.0.4.101", "nodes": ["rx1"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.2.1", "nodes": ["router"]},
 {"addr": "10.0.1.0/24", "gw": "10.0.2.1", "nodes": ["router"]},

]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Configure Experiment

In [ ]:
 installing the kernel
pkg_list = ['iproute2_5.10.0-1_amd64.deb',
            'iproute2-doc_5.10.0-1_all.deb',
            'linux-headers-5.15.72-43822a283-prague-43_1_amd64.deb',
            'linux-image-5.15.72-43822a283-prague-43_1_amd64.deb',
            'linux-libc-dev_1_amd64.deb']
for node in slice.get_nodes():
    for pkg in pkg_list:
        node.upload_file("/home/fabric/work/setup/" + pkg, "/home/ubuntu/" + pkg)
    node.execute("sudo dpkg -i " + " ".join(pkg_list) + "; sudo reboot")

# wait for all nodes to come back up
slice.wait_ssh(progress=True)
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
#inital configuration for the senders
slice.get_node(name="tx0").execute("sudo modprobe tcp_prague")
slice.get_node(name="tx0").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=prague")
slice.get_node(name="tx0").execute("sudo sysctl -w net.ipv4.tcp_ecn=3")

slice.get_node(name="tx1").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=cubic")
slice.get_node(name="tx1").execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Execute Experiment

In [ ]:
# nodes and instances

tx0_node = slice.get_node(name="tx0")
tx1_node = slice.get_node(name="tx1")
rx0_node = slice.get_node(name="rx0")
rx1_node = slice.get_node(name="rx1")
delay_node = slice.get_node(name="delay")
router_node = slice.get_node(name="router")

# interfaces

tx0_egress_iface  = tx0_node.get_interface(network_name = "net-tx0")
tx1_egress_iface  = tx1_node.get_interface(network_name = "net-tx1")

delay_ingress_tx0_iface  = delay_node.get_interface(network_name = "net-tx0")
delay_ingress_tx1_iface  = delay_node.get_interface(network_name = "net-tx1")
delay_egress_iface  = delay_node.get_interface(network_name = "net-delay-router")
delay_ingress_tx0_name = delay_ingress_tx0_iface.get_device_name()
delay_ingress_tx1_name = delay_ingress_tx1_iface.get_device_name()
delay_egress_name = delay_egress_iface.get_device_name()

router_ingress_iface  = router_node.get_interface(network_name = "net-delay-router")
router_egress_rx0_iface  = router_node.get_interface(network_name = "net-rx0")
router_egress_rx1_iface  = router_node.get_interface(network_name = "net-rx1")

router_egress_rx0_name  = router_egress_rx0_iface.get_device_name()
router_egress_rx1_name  = router_egress_rx1_iface.get_device_name()


rx0_ingress_iface  = rx0_node.get_interface(network_name = "net-rx0")
rx1_ingress_iface  = rx1_node.get_interface(network_name = "net-rx1")

In [ ]:
# generate full factorial experiment
import itertools

exp_factors = { 
    'n_bdp': [0.5, 2, 5, 10], # n x bandwidth delay product
    'btl_capacity': [100, 1000],
    'base_rtt': [10, 50, 100],
    'aqm': ['FIFO', 'single queue FQ', 'Codel', 'FQ', 'FQ_Codel', 'DualPI2'],
    'ecn_threshold': [5, 20],
    'ecn_fallback': [0, 1], # 0: OFF, 1: ON
    'rx0_ecn': [0, 1, 2], #0: noecn, 1:ecn, 2:accecn
    'rx1_ecn': [0, 1, 2]  #0: noecn, 1:ecn, 2:accecn

}

factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []
for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    if temp_dict['aqm'] == 'FIFO':
        del temp_dict['ecn_threshold']
    exp_lists.append(temp_dict)

data_dir = slice_name + 'singlebottleneck'

In [ ]:
# run experiments
import time
d = 2 #duration in seconds

em = [delay_ingress_tx0_name, delay_ingress_tx1_name, delay_egress_name]

commands_noecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=cubic  
sudo sysctl -w net.ipv4.tcp_ecn=0''' #tcp_prague, no ECN

commands_ecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=cubic  
sudo sysctl -w net.ipv4.tcp_ecn=1''' #tcp_prague, ECN

commands_accecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=prague  
sudo sysctl -w net.ipv4.tcp_ecn=3''' #tcp_prague, AccECN

server_ecn_list=[commands_noecn, commands_ecn, commands_accecn]

for exp in exp_lists:

    # check if we already ran this experiment
    # (allow stop/resume)
    name_prague="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d.txt" % ("prague",exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
    name_cubic="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d.txt" % ("cubic",exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
    
    file_out = data_dir_tx0 + "-"+ name_prague
    stdout, stderr = tx0_node.execute("ls " + file_out, quiet=True) # run this on the node that saves the output file #write 4 lines

    if len(stdout):
        print("Already have " + file_out + ", skipping")

    elif len(stderr):
        print("Running experiment to generate " + file_out) 
        
        # delay at emulator
        for e in em:
            cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=exp['base_rtt']/2)
            delay_node.execute(cmds)
            print("successfull")
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*2*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1
        
        
        #ecn-fallback configuration
               
        commands = "cd /sys/module/tcp_prague/parameters && echo {value} | sudo tee prague_ecn_fallback".format(value=str(exp['ecn_fallback']))
        tx0_node.execute(commands)
        
        #receiver ecn configuration
        rx0_node.execute(server_ecn_list[exp['rx0_ecn']])
        rx1_node.execute(server_ecn_list[exp['rx1_ecn']])
        
        #aqm type selection
        if exp['aqm']=='FIFO':
            cmds = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            sudo tc qdisc add dev {iface} parent 1:3 handle 3: bfifo limit {buffer} 
            '''.format(iface=router_ingress_name, capacity=exp['btl_capacity'], buffer=btl_limit)
            router_node.execute(cmds)
        
        elif exp['aqm']=='single queue FQ':
            cmds = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit
            sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 ce_threshold {threshold}ms
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], packet_limit=packet_number, threshold=exp['ecn_threshold'])
            router_node.execute(cmds)
            
            
        elif exp['aqm']=='Codel':
            cmds = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit
            sudo tc qdisc replace dev {iface} parent 1:3 handle 3: codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp['ecn_threshold'])
            router_node.execute(cmds)
            
            
        elif exp['aqm']=='FQ':
            cmds = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit
            sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], packet_limit=packet_number, threshold=exp['ecn_threshold'])
            router_node.execute(cmds)
            
            
        elif exp['aqm']=='FQ_Codel':
            cmds = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit
            sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq_codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp['ecn_threshold'])
            router_node.execute(cmds)
        
        elif exp['aqm']=='DualPI2':
            cmds = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit
            sudo tc qdisc add dev {iface} parent 1:3 handle 3: dualpi2 target {threshold}ms
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], threshold=exp['ecn_threshold'])
            router_node.execute(cmds)
            
    
        #starting experiment
        #name_prague="/%s_%0.1f_%d_%d_%s_%s_%s_%s_%s.txt" % ("prague",exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])
        #name_cubic="/%s_%0.1f_%d_%d_%s_%s_%s_%s_%s.txt" % ("cubic",exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx0_ecn'], exp['rx1_ecn'])

        rx0_node.execute("killall iperf3")
        rx1_node.execute("killall iperf3")
        
        ss_tx0_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.3.100 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"
        ss_tx1_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.4.100 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"

        #print("Starting experiment with {1} bdp {2} capacity {3} rtt {4} {5} thrshold {6} ecn_fallback {7} rx0 {8} rx1 for {duration} seconds".format(duration=d, 1=exp['n_bdp'], 2=exp['btl_capacity'], 3=exp['base_rtt'], 4=exp['aqm'], 5=exp['ecn_threshold'], 6= exp['ecn_fallback'], 7=exp['rx0_ecn'], 8=exp['rx1_ecn']))
        
        rx0_node.execute("iperf3 -s -1 -p 4000 -D")
        rx1_node.execute("iperf3 -s -1 -p 5000 -D")
        
        tx0_node.execute_thread(ss_tx0_script.format(flow=name_prague, duration=d))
        tx1_node.execute_thread(ss_tx1_script.format(flow=name_cubic, duration=d))
        
        tx0_node.execute_thread("sleep 1; iperf3 -c 10.0.3.100 -t {duration} -P {flows} -C prague -p 4000 -J > {flow}-result.json".format(flow =name_prague, duration=d, flows=1))
        stdout, stderr = tx1_node.execute("sleep 1; iperf3 -c 10.0.4.100 -t {duration} -P {flows} -C cubic -p 5000 -J > {flow}-result.json".format(flow =name_cubic, duration=d, flows=1))
        time.sleep(3)
        
        break
        
tx0_node.execute('rm -r '+data_dir_tx0)
tx0_node.execute('mkdir '+data_dir_tx0)

tx0_node.execute('mv *.json '+ data_dir_tx0)
tx0_node.execute('mv *.txt '+ data_dir_tx0)
        
tx0_node.execute('tar -czvf '+data_dir_tx0+ '.tgz ' +  data_dir_tx0)
tx0_node.download_file(data_dir_tx0+'.tgz ', '/home/ubuntu/' + data_dir_tx0+ '.tgz')

tx1_node.execute('rm -r '+data_dir_tx1)
tx1_node.execute('mkdir '+data_dir_tx1)

tx1_node.execute('mv *.json '+ data_dir_tx1)
tx1_node.execute('mv *.txt '+ data_dir_tx1)
        
tx1_node.execute('tar -czvf '+data_dir_tx1+ '.tgz ' +  data_dir_tx1)
tx1_node.download_file(data_dir_tx1+'.tgz ', '/home/ubuntu/' + data_dir_tx1+ '.tgz')